In [ ]:
# If PEPit is not installed yet, you can run this cell.
!pip install pepit

# Code description

In this notebook, we numerically verify Lemma C.6 from 

> "Convergence of Proximal Point and Extragradient-Based Methods Beyond Monotonicity: the Case of Negative Comonotonicity"

## Problem

We consider the following problem
       $$ \text{find } x^\ast \text{ such that }F(x^\ast) = 0$$
where $F:\mathbb{R}^d \to \mathbb{R}^d$ is a maximal $\rho$-star negatively comonotone and $L$-Lipschitz operator.

## Algorithm

We consider optimistic gradient method (OG) with $\tilde{x}^0 = x^0$ and for all $k > 0$
$$
\tilde{x}^k = x^k - \gamma_1 F(\tilde{x}^{k-1}),
$$
$$
x^{k+1} = x^k - \gamma_2 F(\tilde{x}^k)
$$

## The goal

Let $k \geq 1$, $\rho \geq 0$ and $\gamma_1,\gamma_2\geq0$; Lemma C.6 states that
$$
\|x^{k+1}-x^*\|^2\leq \|x^k-x^*\|- \gamma_2\left(\gamma_1 - 2\rho - \gamma_2\right)\|F(\tilde{x}^k)\|^2 - \gamma_1\gamma_2\|F(\tilde{x}^{k-1})\|^2  + \gamma_1\gamma_2\|F(\tilde{x}^k) - F(\tilde{x}^{k-1})\|^2. 
$$


Below we verify this inequality numerically.

In [1]:
# Run this code before executing the cell below
from math import sqrt
import numpy as np
from PEPit import PEP

from PEPit.function import Function


class LipschitzStarNegativeComonotoneOperator(Function):
    
    def __init__(self,
                 rho,
                 L=1.,
                 is_leaf=True,
                 decomposition_dict=None,
                 reuse_gradient=True):
        
        super().__init__(is_leaf=is_leaf,
                         decomposition_dict=decomposition_dict,
                         reuse_gradient=True)
        # Store L and rho
        self.rho = rho
        self.L = L

        if self.L == np.inf:
            print("\033[96m(PEPit) The class of Lipschitz star negative comonotone operators is necessarily continuous.\n"
                  "To instantiate an operator, please avoid using the class LipschitzNegativeComonotoneOperator with\n"
                  " L == np.inf. Instead, please use the class NegativeComonotoneOperator (which accounts for the fact\n"
                  "that the image of the operator at certain points might not be a singleton).\033[0m")

    def add_class_constraints(self):
        for point_i in self.list_of_points:

            xi, gi, fi = point_i

            for point_j in self.list_of_points:

                xj, gj, fj = point_j

                if (xi != xj) | (gi != gj):
                    # Interpolation conditions of Lipschitz operator class
                    self.add_constraint((gi - gj)**2 - self.L**2 * (xi - xj)**2 <= 0)
                    
        for point_i in self.list_of_stationary_points:

            xi, gi, fi = point_i

            for point_j in self.list_of_points:

                xj, gj, fj = point_j

                if point_i != point_j:
                    # Interpolation conditions of star negative comonotone operator class
                    self.add_constraint(gj * (xj-xi) + self.rho *  gj ** 2 >= 0)

In [2]:
##########################################################
# parameters: MODIFY HERE!

# pick the parameters for which you want to verify
# the inequality (numerically)
L = .2;
gamma1 = 100.0 ;
gamma2 = 10;
rho = 200;


# verbose & verification tolerance options
verbose = 0;
tolerance = 1e-5;

##########################################################



# (0) Initialize an empty PEP
problem = PEP()

# (1) Set up the problem class
L  =  L; rho = rho; # F is 1-Lipschitz and \rho-negative comonotone
F  = problem.declare_function(LipschitzStarNegativeComonotoneOperator, L=L, rho=rho)
xs = F.stationary_point();  # x^* is a solution

# (2) Set up the starting points
tx0 = problem.set_initial_point() # this is \tilde{x}^0
x1  = problem.set_initial_point() # this is x^1

# (3) Run the algorithm
tx1     = x1 - gamma1 * F.gradient(tx0); 
x2      = x1 - gamma2 * F.gradient(tx1);

# (3) define the expressions (recall that our objective is to verify that
#     psi2 - psi1 - residual<=0 for all F and sequence generated by the
#     optimistic gradient method.

psi2 = (x2-xs)**2;
psi1 = (x1-xs)**2;
term1 = -gamma2*(gamma1-2*rho-gamma2)*(F.gradient(tx1))**2
term2 = -gamma1*gamma2* (F.gradient(tx0))**2
term3 = gamma1*gamma2* (F.gradient(tx1)-F.gradient(tx0))**2
residual = term1+term2+term3

# (4) Set up the performance measure:
expression_to_verify = psi2 - psi1 - residual;
problem.set_performance_metric(expression_to_verify);

# (5) Solve the PEP
worstcase_value = problem.solve(verbose=verbose)


# (6) is the potential verified? Success if expression2 - expression1 <= 0 for the 
#     choice of the parameters above.
print('Did PEPit verify the potential (within prescribed numerical precision)? {:}  \t'.format(worstcase_value<tolerance))
worstcase_value # this should be close to zero

Did PEPit verify the potential (within prescribed numerical precision)? True  	


3.7655434326211434e-12